# Curso: Bioestatística — Experimento Fatorial DIC (Delineamento Inteiramente Casualizado)
## Autores: Sandro da Silva Camargo e Fernando Cardoso

**Problema**: Um pesquisador pretende avaliar o efeito da irrigação e da calagem sobre a produção de uma determinada cultura (variável resposta
Y, expressa em kg de plantas por hectare). A hipótese é que tanto a irrigação quanto a calagem, isoladamente ou em conjunto, podem influenciar o rendimento da cultura.

Fatores estudados:
* Irrigação (I)
- I₀: sem irrigação

- I₁: com irrigação
* Calagem (C)
- C₀: sem calagem

- C₁: com calagem

Estrutura experimental:
Para estudar simultaneamente os efeitos desses dois fatores e sua possível interação, o pesquisador planejou um experimento fatorial 2×2, resultando em quatro combinações de tratamentos:

|Tratamento|Irrigação|Calagem|
|:--|:--:|:--:|
|I₀C₀	|Sem irrigação	|Sem calagem|
|I₀C₁	|Sem irrigação	|Com calagem|
|I₁C₀	|Com irrigação	|Sem calagem|
|I₁C₁	|Com irrigação	|Com calagem|

Cada tratamento será aplicado a um número igual de unidades experimentais (por exemplo, parcelas), permitindo comparar as médias de produção entre os grupos.

Objetivo:
*  Verificar os efeitos principais da irrigação e da calagem sobre a produção;
*  Verificar a interação entre os fatores, isto é, se o efeito de um fator depende da presença do outro.

A base de dados está disponível [aqui](https://raw.githubusercontent.com/Sandrocamargo/biostatistics/refs/heads/master/datasets/fatorial-dic-dados.txt).

Abra este código no seu google colab [clicando aqui](https://colab.research.google.com/github/Sandrocamargo/biostatistics/blob/master/python/bioe_05_fatorial_dic.ipynb).

In [ ]:
# %% Importação de pacotes
!pip install pingouin
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy import stats
import pingouin as pg

In [ ]:
# %% Carregar dados
dados = pd.read_table("https://raw.githubusercontent.com/Sandrocamargo/biostatistics/refs/heads/master/datasets/fatorial-dic-dados.txt")

# Converter colunas em fatores (categorias)
dados["TR"] = dados["TR"].astype("category")   # Tratamento
dados["IR"] = dados["IR"].astype("category")   # Irrigação
dados["CAL"] = dados["CAL"].astype("category") # Calagem

# O grupo de controle é o primeiro desta lista
dados["IR"] = pd.Categorical(
    dados["IR"],
    categories=["Sem", "Com"],
    ordered=True
)
dados["CAL"] = pd.Categorical(
    dados["CAL"],
    categories=["Sem", "Com"],
    ordered=True
)
dados.info()


In [ ]:
# %% Boxplots para verificar outliers
plt.figure()
dados.boxplot(column="Y", by="IR")
plt.title("Boxplot por Irrigação")
plt.suptitle("")
plt.xlabel("Irrigação")
plt.ylabel("Y")
plt.show()

plt.figure()
dados.boxplot(column="Y", by="CAL")
plt.title("Boxplot por Calagem")
plt.suptitle("")
plt.xlabel("Calagem")
plt.ylabel("Y")
plt.show()

In [ ]:
# %% Modelo fatorial com interação
modelo = smf.ols("Y ~ C(IR) * C(CAL)", data=dados).fit()
print(modelo.summary())
print("\n=== ANOVA - Modelo fatorial ===")
print(anova_lm(modelo, typ=2))

# 🎯 Objetivo do modelo

Avaliar os efeitos principais de irrigação (IR) e calagem (CAL), e o efeito de interação IR×CAL sobre a produção média (Y).

# 📈 Qualidade do ajuste

*  R² = 0.951 → O modelo explica 95,1% da variação total da produção.

*  R² ajustado = 0.932 → Mesmo considerando o número de preditores, o ajuste continua muito bom.

*  F(3,8) = 51.56; p < 0.001 → O modelo global é altamente significativo, ou seja, pelo menos um dos fatores (ou sua interação) afeta Y.

# ⚙️ Coeficientes estimados
|Termo|	Interpretação|	Coef.|	p-valor|	Significância|
|:--|:--|:--|:--|:--|
|Intercepto |	Média do tratamento controle (sem irrigação e sem calagem).	| 28.0	| $<$0.001	| ✅ significativo|
|C(IR)[T.Com] |	A irrigação, isoladamente, aumenta em média 10 unidades a produção.	| 10.0 | 0.010	|✅ significativo	|
|C(CAL)[T.Com] |A calagem, isoladamente, aumenta em média 4 unidades, mas o efeito não é significativo.	| 4.0 |0.219	|❌ não significativo	|
|Interação (IR×CAL) |Quando ambas são aplicadas, há um ganho adicional de 20 unidades (efeito conjunto), além dos efeitos individuais.|20.0	|0.002	|✅ altamente significativo|

# Interpretação dos resultados

Efeito principal da Irrigação (IR):

*  F = 88.89, p < 0.001 → altamente significativo.

**Conclusão**: a irrigação tem efeito significativo sobre Y, ou seja, a média de Y difere entre os níveis de irrigação.

Efeito principal da Calagem (CAL):

*  F = 43.56, p < 0.001 → também altamente significativo.

**Conclusão**: a calagem tem efeito significativo sobre Y, a média de Y difere entre os níveis de calagem.

Interação IR × CAL:

*  F = 22.22, p = 0.0015 → significativa.

**Conclusão**: o efeito de irrigação depende do nível de calagem, e vice-versa.

Isso indica que não podemos interpretar apenas os efeitos principais isoladamente; é necessário analisar os desdobramentos da interação.

Resíduo:

*  Soma de quadrados = 108, df = 8

**Conclusão**: Representa a variação não explicada pelos fatores e interação, ou seja, o erro experimental.

#📌 Síntese

*  Todos os fatores (IR, CAL) e a interação entre eles são estatisticamente significativos.

O próximo passo seria analisar os efeitos simples, ou seja:
*  Comparar níveis de IR dentro de cada nível de CAL
*  Comparar níveis de CAL dentro de cada nível de IR

Isso ajuda a entender como cada fator age dependendo do outro.

In [ ]:
# %% Gráfico de interação
fig, ax = plt.subplots()
pd.DataFrame(dados).pivot_table(index="IR", columns="CAL", values="Y", observed=False).plot(ax=ax, marker="o")
plt.title("Interação: Irrigação x Calagem")
plt.ylabel("Kg de planta por parcela")
plt.show()

In [ ]:
# %% Resíduos e diagnósticos
residuos = modelo.resid
print("\nShapiro-Wilk Test (normalidade):", stats.shapiro(residuos))
plt.hist(residuos, bins=8)
plt.title("Histograma dos resíduos")
plt.xlabel("Resíduo")
plt.ylabel("Frequência")
plt.show()

# Homocedasticidade (Levene)
print("\nLevene Test (homogeneidade):")
print(stats.levene(*[group["Y"].values for name, group in dados.groupby("TR")]))

# Independência dos erros (Durbin-Watson)
dw = sm.stats.durbin_watson(residuos)
print(f"\nDurbin-Watson = {dw:.3f}")

In [ ]:
# %% Teste de Tukey (tratamentos)
tukey = pairwise_tukeyhsd(dados["Y"], dados["TR"])
print("\n=== Tukey HSD - Tratamentos ===")
print(tukey.summary())

# Gráfico do Tukey
means = dados.groupby("TR", observed=False)["Y"].mean()
plt.bar(means.index, means.values, color="skyblue")
plt.title("Tukey HSD - Médias por Tratamento")
plt.xlabel("Tratamento")
plt.ylabel("Média de Y")
plt.show()

# %% Testes de Tukey para os fatores principais
print("\n=== Tukey HSD - Irrigação ===")
tukey_ir = pairwise_tukeyhsd(dados["Y"], dados["IR"])
print(tukey_ir.summary())

print("\n=== Tukey HSD - Calagem ===")
tukey_cal = pairwise_tukeyhsd(dados["Y"], dados["CAL"])
print(tukey_cal.summary())

# %% Análise dos efeitos simples (desdobramento da interação)
# Estudar IR dentro de cada nível de CAL (I/C)
print("\n=== Desdobramento: Irrigação dentro de cada nível de Calagem ===")
for cal in dados["CAL"].cat.categories:
    sub = dados[dados["CAL"] == cal]
    modelo_sub = smf.ols("Y ~ C(IR)", data=sub).fit()
    print(f"\nCalagem = {cal}")
    print(anova_lm(modelo_sub, typ=2))

# Estudar CAL dentro de cada nível de IR (C/I)
print("\n=== Desdobramento: Calagem dentro de cada nível de Irrigação ===")
for ir in dados["IR"].cat.categories:
    sub = dados[dados["IR"] == ir]
    modelo_sub = smf.ols("Y ~ C(CAL)", data=sub).fit()
    print(f"\nIrrigação = {ir}")
    print(anova_lm(modelo_sub, typ=2))
